In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.utils import resample
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
exp_raw_data = '/content/drive/MyDrive/MODELLING/Dataset_modelA.xlsx'
collab_data = pd.read_excel(exp_raw_data)

In [ ]:
collab_data.corr().abs()['CrVI_concentration']

In [ ]:
def remove_outliers_iqr(df, column, k=1.5):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - k * IQR
    upper_bound = Q3 + k * IQR
    outliers_removed = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers_removed, outliers

columns_for_outlier_removal = ['CrVI_concentration','initial_concentration','volume','Ph']

outliers_removed_data = collab_data.copy()
outliers_data = pd.DataFrame(columns=collab_data.columns)

for column in columns_for_outlier_removal:
    outliers_removed_data, outliers = remove_outliers_iqr(outliers_removed_data, column)
    outliers_data = pd.concat([outliers_data, outliers])

print('Outliers list length', len(outliers_data))
print('Corrected dataset length', len(outliers_removed_data))

In [ ]:
outliers_removed_data.corr().abs()['CrVI_concentration']

In [ ]:
y = outliers_removed_data['CrVI_concentration']
features = ['initial_concentration','adsorbent_mass','flow_rate','Ph','volume']
X = outliers_removed_data[features]

X_testing, X_train, y_testing, y_train = train_test_split(X, y, test_size=0.7, random_state=27)
X_val, X_test, y_val, y_test = train_test_split(X_testing, y_testing, test_size=0.5, random_state=27)
print("Training set size", len(X_train))
print("Validation set size", len(X_val))
print("Training set size", len(X_test))
drive_path1 = '/content/drive/My Drive/MODELLING/X_testA.csv'
X_test.to_csv(drive_path1, index=False)
drive_path2 = '/content/drive/My Drive/MODELLING/y_testA.csv'
y_test.to_csv(drive_path2, index=False)


In [ ]:
outliers_removed_data['initial_concentration'].plot(kind='kde')
outliers_removed_data['initial_concentration'].plot(kind='hist', density = True)

In [ ]:
outliers_removed_data['adsorbent_mass'].plot(kind='kde')
outliers_removed_data['adsorbent_mass'].plot(kind='hist', density = True)

In [ ]:
outliers_removed_data['flow_rate'].plot(kind='kde')
outliers_removed_data['flow_rate'].plot(kind='hist', density = True)

In [ ]:
hist = outliers_removed_data['CrVI_concentration'].plot(kind='hist', density = True, bins=5)

In [ ]:
train_data = pd.concat([X_train, y_train], axis=1)

majority_class_train = train_data[train_data['CrVI_concentration'] <= 12.5]
minority_class_train = train_data[train_data['CrVI_concentration'] > 12.5]

downsampled_majority_train = resample(majority_class_train,
                                      replace=False,
                                      n_samples=len(minority_class_train),
                                      random_state=27)

balanced_train_data = pd.concat([downsampled_majority_train, minority_class_train])

X_train_balanced = balanced_train_data[features]
y_train_balanced = balanced_train_data['CrVI_concentration']

print("Majority class size:", len(majority_class_train))
print("Minority class size:", len(minority_class_train))
print("Original training dataset size:", len(X_train))
print("Balanced training dataset size:", len(X_train_balanced))

In [ ]:
rf_trail1 = RandomForestRegressor(random_state=27)
rf_trail1.fit(X_train_balanced, y_train_balanced)

rf_trail1_prediction_train = rf_trail1.predict (X_train_balanced)

rf_trail1_mae_TRAIN = mean_absolute_error(rf_trail1_prediction_train, y_train_balanced)
rf_trail1_mape_TRAIN = mean_absolute_percentage_error(rf_trail1_prediction_train, y_train_balanced)
rf_trail1_linearity_TRAIN = r2_score(rf_trail1_prediction_train, y_train_balanced)

print('rf_trail1_mae_TRAIN =', rf_trail1_mae_TRAIN)
print('rf_trail1_mape_TRAIN =', rf_trail1_mape_TRAIN)
print('rf_trail1_linearity_TRAIN =', rf_trail1_linearity_TRAIN)

plt.figure(figsize=(10, 6))
plt.scatter(y_train_balanced, rf_trail1_prediction_train, color='blue', label='Predicted vs Actual')
plt.plot([min(y_train_balanced), max(y_train_balanced)], [min(y_train_balanced), max(y_train_balanced)], color='red', linestyle='--', label='Perfect Prediction Line')
plt.title('Prediction vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()

In [ ]:
rf_trail1_prediction = rf_trail1.predict (X_val)
rf_trail1_mae = mean_absolute_error(rf_trail1_prediction, y_val)
rf_trail1_mape = mean_absolute_percentage_error(rf_trail1_prediction, y_val)
rf_trail1_linearity = r2_score(rf_trail1_prediction, y_val)

print('rf_trail1_mae =', rf_trail1_mae)
print('rf_trail1_mape =', rf_trail1_mape)
print('rf_trail1_linearity =', rf_trail1_linearity)

plt.figure(figsize=(10, 6))
plt.scatter(y_val, rf_trail1_prediction, color='blue', label='Predicted vs Actual')
plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], color='red', linestyle='--', label='Perfect Prediction Line')
plt.title('Prediction vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()

In [ ]:
feat_importance_model_rf_trail1 = pd.Series(rf_trail1.feature_importances_, index=X_train_balanced.columns)
feat_importance_model_rf_trail1.nlargest(5).plot(kind='barh')

Hyperparameter optimization (random grid search)

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 10)]
max_features = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
max_depth = [int(x) for x in np.linspace (10, 110, num = 11)]
min_samples_leaf = [1,2,4]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf,
               }

In [ ]:
def custom_score(y_true, y_pred):
    return r2_score(y_true, y_pred)
custom_scorer = make_scorer(custom_score)
rf_trail2 = RandomForestRegressor()
rf_trail2_random = RandomizedSearchCV(estimator=rf_trail2, param_distributions=random_grid,
                              n_iter = 100, scoring=custom_scorer,
                              cv = 5, verbose=2, random_state=35, n_jobs=-1,
                              return_train_score=True)

In [ ]:
rf_trail2_random.fit(X_train_balanced,y_train_balanced)
rf_trail2_prediction = rf_trail2_random.predict (X_train_balanced)

In [ ]:
params = rf_trail1.get_params()
print("Initial Parameters:")
for param_name, param_value in params.items():
    print(f"{param_name}: {param_value}")
print('Best Parameters found: ', rf_trail2_random.best_params_, ' \n' )

In [ ]:
rf_trail2_random_model = RandomForestRegressor(n_estimators = 120, min_samples_leaf = 1, max_features = 0.8, max_depth = 10, random_state = 27)
rf_trail2_prediction_train = rf_trail2_random_model.fit(X_train_balanced, y_train_balanced)
rf_trail2_prediction_train = rf_trail2_random_model.predict(X_train_balanced)

rf_trail2_mae_TRAIN = mean_absolute_error(rf_trail2_prediction_train, y_train_balanced)
rf_trail2_mape_TRAIN = mean_absolute_percentage_error(rf_trail2_prediction_train, y_train_balanced)
rf_trail2_linearity_TRAIN = r2_score(rf_trail2_prediction_train, y_train_balanced)

print('rf_trail2_mae_TRAIN =', rf_trail2_mae_TRAIN)
print('rf_trail2_mape_TRAIN =', rf_trail2_mape_TRAIN)
print('rf_trail2_linearity_TRAIN =', rf_trail2_linearity_TRAIN)

plt.figure(figsize=(10, 6))
plt.scatter(y_train_balanced, rf_trail2_prediction_train, color='blue', label='Predicted vs Actual')
plt.plot([min(y_train_balanced), max(y_train_balanced)], [min(y_train_balanced), max(y_train_balanced)], color='red', linestyle='--', label='Perfect Prediction Line')
plt.title('Prediction vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()

In [ ]:
rf_trail2_prediction = rf_trail2_random_model.predict (X_val)

rf_trail2_mae = mean_absolute_error(rf_trail2_prediction, y_val)
rf_trail2_mape = mean_absolute_percentage_error(rf_trail2_prediction, y_val)
rf_trail2_linearity = r2_score(rf_trail2_prediction, y_val)

print('rf_trail1_mae/rf_trail2_mae =', rf_trail1_mae, rf_trail2_mae)
print('rf_trail1_mape/rf_trail2_mape =', rf_trail1_mape, rf_trail2_mape)
print('rf_trail1_linearity/rf_trail2_linearity =', rf_trail1_linearity, rf_trail2_linearity)

plt.figure(figsize=(10, 6))
plt.scatter(y_val, rf_trail2_prediction, color='blue', label='Predicted vs Actual')
plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], color='red', linestyle='--', label='Perfect Prediction Line')
plt.title('Prediction vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()

Hyperparameter optimization (bayesian optimization)

In [ ]:
!pip install bayesian-optimization scikit-learn pandas
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from bayes_opt import BayesianOptimization
from scipy.stats import pearsonr

In [ ]:
def linearity_score(y_true, y_pred):
       residuals = y_true - y_pred
       corr_coef, _ = pearsonr(y_pred, residuals)
       return np.abs(corr_coef)

def objective(n_estimators, max_depth, min_samples_leaf, max_features):
    rf_trail3= RandomForestRegressor(n_estimators=int(n_estimators),
                                  max_depth=int(max_depth),
                                  min_samples_leaf=int(min_samples_leaf),
                                  max_features=min(max_features, 0.999),
                                  random_state=27)
    y_pred = cross_val_predict(rf_trail3, X_train_balanced, y_train_balanced, cv=5)
    mse = mean_squared_error(y_train_balanced, y_pred)
    linearity = linearity_score(y_train_balanced, y_pred)
    score = -mse - linearity
    return score

In [ ]:
param_bounds = {
    'n_estimators': (10, 1000),
    'max_depth': (1, 100),
    'min_samples_leaf': (1,5),
    'max_features': (0.1, 0.999),
}
optimizer = BayesianOptimization(f=objective, pbounds=param_bounds, random_state=27)
optimizer.maximize(init_points=5, n_iter=15)

In [ ]:
best_params=optimizer.max['params']
best_params

In [ ]:
rf_trail3_bayesian_model = RandomForestRegressor(n_estimators=int(best_params['n_estimators']),
                                   max_depth=int(best_params['max_depth']),
                                   min_samples_leaf=int(best_params['min_samples_leaf']),
                                   max_features=best_params['max_features'],
                                   random_state=27)
rf_trail3_prediction_train = rf_trail3_bayesian_model.fit(X_train_balanced, y_train_balanced)
rf_trail3_prediction_train = rf_trail3_bayesian_model.predict(X_train_balanced)

rf_trail3_mae_TRAIN = mean_absolute_error(rf_trail3_prediction_train, y_train_balanced)
rf_trail3_mape_TRAIN = mean_absolute_percentage_error(rf_trail3_prediction_train, y_train_balanced)
rf_trail3_linearity_TRAIN = r2_score(rf_trail3_prediction_train, y_train_balanced)

print('rf_trail3_mae_TRAIN =', rf_trail3_mae_TRAIN)
print('rf_trail3_mape_TRAIN =', rf_trail3_mape_TRAIN)
print('rf_trail3_linearity_TRAIN =', rf_trail3_linearity_TRAIN)

plt.figure(figsize=(10, 6))
plt.scatter(y_train_balanced, rf_trail3_prediction_train, color='blue', label='Predicted vs Actual')
plt.plot([min(y_train_balanced), max(y_train_balanced)], [min(y_train_balanced), max(y_train_balanced)], color='red', linestyle='--', label='Perfect Prediction Line')
plt.title('Prediction vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()

In [ ]:
rf_trail3_prediction = rf_trail3_bayesian_model.predict (X_val)

rf_trail3_mae = mean_absolute_error(rf_trail3_prediction, y_val)
rf_trail3_mape = mean_absolute_percentage_error(rf_trail3_prediction, y_val)
rf_trail3_linearity = r2_score(rf_trail3_prediction, y_val)

print('rf_trail2_mae/rf_trail3_mae =', rf_trail2_mae, rf_trail3_mae)
print('rf_trail2_mape/rf_trail3_mape =', rf_trail2_mape, rf_trail3_mape)
print('rf_trail2_linearity/rf_trail3_linearity =', rf_trail2_linearity, rf_trail3_linearity)

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(y_val, rf_trail3_prediction, color='blue', label='Predicted vs Actual')
plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], color='red', linestyle='--', label='Perfect Prediction Line')
plt.title('Prediction vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()

Final model saving

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from joblib import dump
rf_optimized_modelA_CrVI = rf_trail3_bayesian_model
drive_path = '/content/drive/My Drive/MODELLING/rf_optimized_modelA_CrVI.joblib'
dump(rf_optimized_modelA_CrVI, drive_path)
print("Model saved successfully to Google Drive.")